In [1]:
import pandas as pd

reviews = pd.read_csv("reviews_with_foods_and_filtered_verbs.csv")

len(reviews)

8449

In [2]:
reviews.head()

,user_id,recipe_id,date,rating,review,foods,verbs
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...,"{'cayenne pepper', 'shake', 'low fat sour crea...","['cook', 'add', 'use']"
1,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin...",{'black pepper'},"['make', 'add']"
2,52282,120345,2005-05-21,4,very very sweet. after i waited the 2 days i b...,"{'cake', 'raspberry'}","['wait', 'buy', 'add', 'go', 'add', 'fill', 't..."
3,2000192946,120345,2015-05-10,2,This recipe was OVERLY too sweet. I would sta...,"{'sugar', 'raspberry'}","['start', 'add', 'have', 'go', 'buy', 'make', ..."
4,136726,197160,2006-11-25,5,I used this mix to make meat balls.Very simple...,"{'garlic', 'sub sandwich', 'cheese', 'sauces',...","['use', 'make', 'put', 'make', 'put', 'find', ..."


In [3]:
recipes = pd.read_csv("RAW_recipes.csv")

len(recipes)

231637

In [4]:
# rename id to recipe_id

recipes = recipes.rename(columns={"id": "recipe_id"})

In [5]:
# take ingredients column from recipes and add it to reviews where id matches

reviews_with_ingredients = pd.merge(reviews, recipes[["recipe_id", "ingredients"]], on="recipe_id")

In [18]:
reviews_with_ingredients.tail()

,user_id,recipe_id,date,rating,review,foods,verbs,ingredients
8444,2001277,3668,2011-09-06,5,This recipe is an unexpected pleasure -- I had...,"{'garlic', 'cheese', 'sherry', 'zucchini', 'sq...","['have', 'come', 'use', 'take', 'mix', 'need',...","['chicken', 'zucchini', 'onion', 'carrot', 'cr..."
8445,1840647,155135,2011-03-02,0,A slight variation on this recipe I've made an...,"{'salsa', 'eggs'}","['make', 'find', 'cut', 'chop', 'get', 'add', ...","['eggs', 'part-skim ricotta cheese', 'plum tom..."
8446,83093,155135,2007-09-09,5,Teeheehee ... I feel so sneaky. \r\n;-) I mad...,"{'eggs', 'ricotta'}","['feel', 'make', 'notice', 'add', 'bake', 'get...","['eggs', 'part-skim ricotta cheese', 'plum tom..."
8447,69927,57122,2004-11-10,2,"I'm sorry, this simply didn't suit my family's...","{'canned pimiento', 'chicken', 'broccoli', 'ga...","['suit', 'like', 'have', 'make', 'use', 'find'...","['fresh broccoli', 'boneless skinless chicken ..."
8448,199792,158715,2008-03-15,5,Yum! The star of the show!! My steaks were qui...,"{'sauces', 'steak'}","['choose', 'think', 'throw', 'reduce', 'change...","['sirloin steak', 'shallot', 'garlic cloves', ..."


In [36]:
reviews_with_ingredients.review[234]

'YUMMY CREAMY FRUITY DELICIOUS! I made this with orange juice and added fresh pineapple chunks. I also used extra ice and added Splenda. Freddy Cat says thanks for the drink! Made for the ZWT4.'

In [47]:
reviews_with_ingredients.ingredients[234]

"['ice', 'banana', 'strawberries', 'pineapple juice', 'coconut milk']"

In [126]:
def make_user_prompt(review, ingredients):

    return "\nReview: " + review + "\nIngredients: " + str(ingredients)

In [69]:

user_example = make_user_prompt(reviews_with_ingredients.review[234], reviews_with_ingredients.ingredients[234])
print(user_example)


Review: YUMMY CREAMY FRUITY DELICIOUS! I made this with orange juice and added fresh pineapple chunks. I also used extra ice and added Splenda. Freddy Cat says thanks for the drink! Made for the ZWT4.
Ingredients: ['ice', 'banana', 'strawberries', 'pineapple juice', 'coconut milk']


In [70]:
system_example_response = str([{"o":"pineapple juice", "n": "orange juice"}, {"o": "-", "n": "fresh pineapple chunks"}, {"o": "-", "n": "ice"}, {"o": "-", "n": "Splenda"}])
print(system_example_response)

[{'o': 'pineapple juice', 'n': 'orange juice'}, {'o': '-', 'n': 'fresh pineapple chunks'}, {'o': '-', 'n': 'ice'}, {'o': '-', 'n': 'Splenda'}]


In [74]:
reviews_with_ingredients.review[3640], reviews_with_ingredients.ingredients[3640]

("Excellent soup. Stone Soup made for a wonderful class project a few years back, one I have always wanted to make a home project. With the making of your recipe -  the soup and 'the story'  I suspect now a 'tradition in the making' at our house. Went with a potato for the symbolic stone and made as posted, the end result was soup that we enjoyed. Think the addition of chicken or turkey as previous reviewers added would work well with this recipe as would rice - making for a good many variations for this old fashioned soup. To round out the meal made Chef TotalFark's corn meal muffins using Recipe#203214. Thank you for sharing both the story and the recipe Sydney Mike!",
 "['butter', 'onion', 'celery ribs', 'carrot', 'red potatoes', 'red bell pepper', 'garlic clove', 'chicken broth', 'zucchini', 'yellow squash', 'corn kernel', 'cooked pasta', 'salt', 'pepper', 'parmesan cheese', 'crouton']")

In [155]:
from openai import OpenAI

# read api key from file

api_key = open("openai_api_key.txt", "r").read()
client = OpenAI(api_key=api_key)
def get_replacements(review, ingredients):

  
  completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
      {"role": "system", "content": "Input: a user review and a list of ingredients. Find all mentions of changes in ingredients list from the review. Output in the following JSON format: {'repl': [{'o': original_ingredient, 'n': new_ingredient}, ...]'}. If original ingredient could not be identified with 100% certainty, leave '-'. If no changes, return empty list."},
      {"role": "user", "content":  make_user_prompt(review, ingredients)},
    ],
    temperature=0,
    response_format={"type": "json_object"},
    seed=42,
    frequency_penalty=0.1,
    n=1,
    max_tokens=1000,
  )

  return completion

completion = get_replacements(reviews_with_ingredients.review[3640], reviews_with_ingredients.ingredients[3640])

In [156]:
completion

ChatCompletion(id='chatcmpl-8c38Oot93oKbG7Ul3FpkTNS1T2jEq', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "repl": [\n    {"o": "-", "n": "chicken"},\n    {"o": "-", "n": "turkey"},\n    {"o": "-", "n": "rice"}\n  ]\n}', role='assistant', function_call=None, tool_calls=None))], created=1704079204, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_772e8125bb', usage=CompletionUsage(completion_tokens=45, prompt_tokens=236, total_tokens=281))

In [101]:
json.dumps({"system_fingerprint": completion.system_fingerprint, "replacements": json.loads(completion.choices[0].message.content)["repl"]})

'{"system_fingerprint": "fp_772e8125bb", "replacements": [{"o": "red potatoes", "n": "potato"}, {"o": "chicken broth", "n": "chicken"}, {"o": "cooked pasta", "n": "rice"}]}'

In [114]:
reviews_with_ingredients.review[1], reviews_with_ingredients.ingredients[1]

('Made the cheddar bacon topping, adding a sprinkling of black pepper. Yum!',
 "['mayonnaise', 'salsa', 'cheddar cheese', 'refried beans', 'ham', 'green pepper', 'tomatoes', 'cooked bacon', 'pepperoni', 'mozzarella cheese', 'parmesan cheese', 'pizza sauce', 'frozen chopped broccoli']")

In [38]:
completion.choices[0].message.content

'\n{\n  "replacements": [\n    {"old": "-", "new": "orange juice"},\n    {"old": "-", "new": "fresh pineapple chunks"},\n    {"old": "-", "new": "Splenda"}\n  ]\n}'

In [10]:
reviews_with_ingredients["ingredients"][2]

"['raspberries', 'granulated sugar']"

In [63]:
import json
from tqdm import tqdm

In [96]:
import time 

In [ ]:
input_tokens = 0
output_tokens = 0

In [159]:

for i in tqdm(range(5322, 10000)):
  review = reviews_with_ingredients["review"][i]
  ingredients = reviews_with_ingredients["ingredients"][i]

  completion = get_replacements(review, ingredients)
  
  system_fingerprint = completion.system_fingerprint
  replacements = completion.choices[0].message.content

  input_tokens += completion.usage.prompt_tokens
  output_tokens += completion.usage.completion_tokens

  
  # save to file
  with open("replacements7.jsonl", "a") as myfile:

    if "repl" not in json.loads(completion.choices[0].message.content):
      array = []
    else:
      array = json.loads(completion.choices[0].message.content)["repl"]

    myfile.write(json.dumps({"index":i,"system_fingerprint": completion.system_fingerprint, "replacements": array}) + "\n")

  time.sleep(3)


 67%|██████▋   | 3127/4678 [4:06:05<2:02:03,  4.72s/it]


KeyError: 8449

In [158]:
completion

ChatCompletion(id='chatcmpl-8c6kEeC3jmUZ7zIhneLqdOTYFB7Pd', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "repl": [\n    {"o": "-", "n": "enchilada sauce"},\n    {"o": "green cho', role='assistant', function_call=None, tool_calls=None))], created=1704093082, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_772e8125bb', usage=CompletionUsage(completion_tokens=29, prompt_tokens=222, total_tokens=251))

In [134]:
completion

ChatCompletion(id='chatcmpl-8brjb4GzoJxaFdvVCoLn5YUsRS6RC', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{}\n\n', role='assistant', function_call=None, tool_calls=None))], created=1704035383, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_772e8125bb', usage=CompletionUsage(completion_tokens=1, prompt_tokens=195, total_tokens=196))

In [97]:
reviews_with_ingredients["review"][6]

"This was a lovely Morrocan style dish. I halved the recipe and made it on the stove top like a stew. I added a bit more stock to keep it from drying out. I left off the parsley and/or cilantro, didn't have any. Served with some warm pita type flat breads. I have a VERY picky Husband that even enjoyed this dish."

In [65]:
completion.choices[0].message.content

'\n{\n  "0": {"o": "2 cups", "n": "-"},\n  "1": {"o": "1/3 cup of sugar", "n": "-"},\n  "2": {"o": "1/4 cup of sugar", "n": "-"}\n}'

In [160]:
estimated_bill = input_tokens / 1000 * 0.001 + output_tokens / 1000 * 0.002
estimated_bill

2.256391